In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 
n
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
pd.options.display.max_rows=150
pd.options.display.max_columns=100
pd.options.display.max_colwidth=200
import datetime as dt
import warnings
warnings.filterwarnings(action='ignore')
from sklearn import preprocessing
from sklearn.metrics import roc_auc_score,f1_score
from sklearn.model_selection import KFold,GroupKFold,ShuffleSplit,StratifiedKFold
import gc
import lightgbm as lgb
from tqdm import tqdm_notebook as tqdm
from imblearn.over_sampling import SMOTE
import mlcrate as mlc

In [ ]:
path = '/kaggle/input/wns-wizard/'

In [ ]:
#Reduce the memory usage - Inspired by Panchajanya Banerjee
def reduce_mem_usage(df, verbose=True):
    numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
    start_mem = df.memory_usage().sum() / 1024**2    
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type in numerics:
            c_min = df[col].min()
            c_max = df[col].max()
            if str(col_type)[:3] == 'int':
                if c_min > np.iinfo(np.int8).min and c_max < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif c_min > np.iinfo(np.int16).min and c_max < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif c_min > np.iinfo(np.int32).min and c_max < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif c_min > np.iinfo(np.int64).min and c_max < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)  
            else:
                if c_min > np.finfo(np.float16).min and c_max < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif c_min > np.finfo(np.float32).min and c_max < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)    
    end_mem = df.memory_usage().sum() / 1024**2
    if verbose: print('Mem. usage decreased to {:5.2f} Mb ({:.1f}% reduction)'.format(end_mem, 100 * (start_mem - end_mem) / start_mem))
    return df


In [ ]:
train_df = reduce_mem_usage(pd.read_csv(f'{path}train.csv',parse_dates=['impression_time',],dayfirst=True))
test_df = reduce_mem_usage(pd.read_csv(f'{path}test.csv',parse_dates=['impression_time',],dayfirst=True))
item_data = reduce_mem_usage(pd.read_csv(f'{path}item_data.csv'))
view_log = reduce_mem_usage(pd.read_csv(f'{path}view_log.csv',parse_dates=['server_time',],dayfirst=True))

In [ ]:
_ = view_log.groupby(['user_id'])['device_type'].unique().reset_index()
_['device_encode']=-1
for k,i in enumerate(tqdm( _['device_type'])):
    s = ''
    for ii in i:
        if ii=='android':
            s+='0'
        elif ii=='web':
            s='1'
        else:
            s+='2'
    s = ''.join(sorted(s,reverse=True))
    _.ix[k,'device_encode']=int(s)

_.drop(columns='device_type',inplace=True)
_.ix[_['device_encode']==10,'device_encode']=1
view_log=pd.merge(view_log,_,how='left',on='user_id')


In [ ]:
view_log['device_encode'].isnull().sum()

In [ ]:
from collections import Counter

def get_class_weights(y):
    counter = Counter(y)
    majority = max(counter.values())
    return  {cls: round(float(majority)/float(count), 2) for cls, count in counter.items()}

class_weights = get_class_weights(train_df['is_click'])
print(class_weights)

In [ ]:
#view_log['device_encode'] = view_log['device_encode'].astype(np.int8)

In [ ]:
train_df['is_click'].value_counts()

In [ ]:
train_df.shape

In [ ]:
## is_click==1
10862/237609

In [ ]:
train_df.head(100)

In [ ]:
test_df.head(100)

In [ ]:
view_log.head(100)

In [ ]:
train_df['is_train']=1
test_df['is_train']=0

In [ ]:
date_cols = ['impression_time']
for date_col in date_cols:
    train_df[date_col + "_in_seconds"] = (train_df[date_col] - dt.datetime(2000,1,1)).dt.total_seconds()
    test_df[date_col + "_in_seconds"] = (test_df[date_col] - dt.datetime(2000,1,1)).dt.total_seconds()
    
    train_df[date_col + "_month"] = train_df[date_col].dt.month
    test_df[date_col + "_month"] = test_df[date_col].dt.month
    
    train_df[date_col + "_hour"] = train_df[date_col].dt.hour
    test_df[date_col + "_hour"] = test_df[date_col].dt.hour
    
    #train_df[date_col + "_year"] = train_df[date_col].dt.year
    #test_df[date_col + "_year"] = test_df[date_col].dt.year
    
    train_df[date_col + "_week"] = train_df[date_col].dt.week
    test_df[date_col + "_week"] = test_df[date_col].dt.week
    
    train_df[date_col + "_weekDay"] = train_df[date_col].dt.weekday
    test_df[date_col + "_weekDay"] = test_df[date_col].dt.weekday

In [ ]:
train_df['is_click'] = train_df['is_click'].astype(np.int8)

In [ ]:
train_df.head()

In [ ]:
dist = pd.DataFrame(np.zeros((train_df['impression_time_hour'].nunique(),4)),columns=['Var','Shape','Zeros','Ones'])
for i,var in enumerate(train_df['impression_time_hour'].unique()):
    mask = train_df['impression_time_hour']==var
    size = train_df[mask].shape[0]
    mask0 = train_df['is_click']==0
    mask1 = train_df['is_click']==1
    ones = train_df[mask & mask1].shape[0]
    zeros = train_df[mask & mask0].shape[0]
    dist.iloc[i,:]=[var,int(size),int(zeros),int(ones)]
dist.sort_values(by='Ones',ascending=False)

In [ ]:
train_df[train_df['is_click']==1]

In [ ]:
print('Train min/max date: %s / %s' % (train_df.impression_time.min().date(), train_df.impression_time.max().date()))
print('Test  min/max date: %s / %s' % ( test_df.impression_time.min().date(),  test_df.impression_time.max().date()))
print('')
print('Number of days in train: %d' % ((train_df.impression_time.max() - train_df.impression_time.min()).days + 1))
print('Number of days in test:  %d' % (( test_df.impression_time.max() -  test_df.impression_time.min()).days + 1))
print('')
print('Train shape: %d rows' % train_df.shape[0])
print('Test shape: %d rows'  % test_df.shape[0])

## Train_Data Preprocessing

In [ ]:
train_test = pd.concat([train_df,test_df],sort=False,axis=0).reset_index(drop=True)
#del train_df,test_df
gc.collect()

In [ ]:
train_test = train_test.sort_values(by=['impression_time']).reset_index(drop=True)
train_test.head()

In [ ]:
cols = ['after_one_day_samples','prev_one_day_samples']
for col in cols:
    train_test[col]=-1

In [ ]:
for i in tqdm(range(train_test.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    
    
    
    s_next_day = pd.date_range(s[1], freq='1D', periods=2)
    after_one_day =s_next_day[1]
    s_prev_day = pd.date_range(end=s[1], freq='1D', periods=2)
    prev_one_day =s_prev_day[0]
        
        
    after_one_day_samples = train_test[mask & (train_test['impression_time'] >current_time ) & (train_test['impression_time'] <= after_one_day)].shape[0]
    prev_one_day_samples = train_test[mask & (train_test['impression_time'] >prev_one_day ) & (train_test['impression_time'] <= current_time)].shape[0]
    
    train_test.iloc[i,[-2,-1]]=after_one_day_samples,prev_one_day_samples
   

In [ ]:
cols = ['after_one_hour_samples','prev_one_hour_samples',]
for col in cols:
    train_test[col]=-1

In [ ]:
for i in tqdm(range(train_test.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    
    
    
    s_next_hour = pd.date_range(s[1], freq='1H', periods=2)
    #print(s[1],s_next_hour[1])
    after_one_hour =s_next_hour[1]
    s_prev_hour = pd.date_range(end=s[1], freq='1H', periods=2)
    #print(s_prev_hour[0],s[1])
    prev_one_hour =s_prev_hour[0]
        
        
        
    after_one_hour_samples = train_test[mask & (train_test['impression_time'] >current_time ) & (train_test['impression_time'] < after_one_hour)].shape[0]
    prev_one_hour_samples = train_test[mask & (train_test['impression_time'] >prev_one_hour ) & (train_test['impression_time'] < current_time)].shape[0]
    train_test.iloc[i,[-2,-1]]=after_one_hour_samples,prev_one_hour_samples
   

In [ ]:
cols = ['after_one_month_samples','prev_one_month_samples']
for col in cols:
    train_test[col]=-1

In [ ]:
for i in tqdm(range(train_df.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    
    
    
    s_next_month = pd.date_range(s[1], freq='1M', periods=2)
    after_one_month =s_next_month[1]
    s_prev_month = pd.date_range(end=s[1], freq='1M', periods=2)
    prev_one_month =s_prev_month[0]
        
        
    after_one_month_samples = train_test[mask & (train_test['impression_time'] >current_time ) & (train_test['impression_time'] < after_one_month)].shape[0]
    prev_one_month_samples = train_test[mask & (train_test['impression_time'] >prev_one_month ) & (train_test['impression_time'] < current_time)].shape[0]
    #print(after_one_month_samples,prev_one_month_samples)
    train_test.iloc[i,[-2,-1]]=after_one_month_samples,prev_one_month_samples
   

In [ ]:
cols = ['after_all_samples','prev_all_samples']
for col in cols:
    train_test[col]=-1

In [ ]:
for i in tqdm(range(train_df.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    prev_all_samples = train_test[mask & (train_test['impression_time'] <current_time)].shape[0]
    after_all_samples = train_test[mask & (train_test['impression_time'] >current_time)].shape[0]
        #print(after_one_hour_samples,prev_one_hour_samples,after_one_day_samples,prev_one_day_samples,after_one_week_samples,prev_one_week_samples,\
              #after_one_month_samples,prev_one_month_samples,prev_all_samples,after_all_samples)
    train_test.iloc[i,[-2,-1]]=after_all_samples,prev_all_samples
    
   

In [ ]:
cols = ['after_one_week_samples','prev_one_week_samples']
for col in cols:
    train_test[col]=-1
    

In [ ]:
for i in tqdm(range(train_df.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    current_time= s[1]

    mask = train_test['user_id']==s[2]
    mask_1 = train_test.iloc[i,0]
    
    
    
    s_next_week = pd.date_range(s[1], freq='1W', periods=2)
    after_one_week =s_next_week[1]
    s_prev_week = pd.date_range(end=s[1], freq='1W', periods=2)
    prev_one_week =s_prev_week[0]
        
        
    after_one_week_samples = train_test[mask & (train_test['impression_time'] >current_time ) & (train_test['impression_time'] < after_one_week)].shape[0]
    prev_one_week_samples = train_test[mask & (train_test['impression_time'] >prev_one_week ) & (train_test['impression_time'] < current_time)].shape[0]
        
    #print(after_one_week_samples,prev_one_week_samples)
    train_test.iloc[i,[-2,-1]]=after_one_week_samples,prev_one_week_samples
   

In [ ]:
train_test.columns

In [ ]:
train_test.drop(columns=['after_one_hour_samples','after_one_day_samples','after_one_week_samples','after_one_month_samples','after_all_samples'],inplace=True)

In [ ]:
train_test.shape

In [ ]:
def hour_encode(val):
    if val in list(range(2,8)):
        return 0
    elif val in list(range(9,14)):
        return 1
    else:
        return 2

In [ ]:
#train_test['hour_encode'] = train_test['impression_time_hour'].apply(hour_encode)

In [ ]:
train_df.replace(np.inf,np.nan,inplace=True)
test_df.replace(np.inf,np.nan,inplace=True)
train_df =  train_df.replace(-np.inf, np.nan)
test_df = test_df.replace(-np.inf, np.nan)

In [ ]:
train_app_unique = train_df['app_code'].unique() 
test_app_unique = test_df['app_code'].unique()
s = [i for i in train_app_unique if i not in test_app_unique]
ss = [i for i in test_app_unique if i not in train_app_unique]

In [ ]:
##unique app_code
train_test.ix[train_test['app_code'].isin(ss+s),'app_code_prsent']=1
train_test.ix[~train_test['app_code'].isin(ss+s),'app_code_prsent']=0
#train_test['app_code_prsent'] = train_test['app_code_prsent'].astype(np.int8)

In [ ]:
## impression_time
_ = train_test.groupby('user_id')['impression_time_in_seconds'].mean().reset_index()
_.columns = ['user_id','user_impression_time_in_second_mean']
train_test = pd.merge(train_test,_,how='left',on='user_id')
train_test["user_cumnum_impression_time"] = train_test.groupby("user_id")["impression_time_in_seconds"].cumcount().values

In [ ]:
_ = train_test.groupby('user_id').size().reset_index()
_.columns = ['user_id','user_id_count']
train_test = pd.merge(train_test,_,how='left',on='user_id')

In [ ]:
train_test['prev_impression_time'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(1)
train_test['time_gap_between_prev'] = train_test['impression_time_in_seconds']-train_test['prev_impression_time']
#train_test['next_impression_time'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(-1)
#train_test['time_gap_between_next'] = train_test['impression_time_in_seconds']-train_test['next_impression_time']
train_test.drop(columns=['prev_impression_time'],inplace=True)

In [ ]:
train_test['prev_impression_time_2'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(2)
train_test['time_gap_between_prev_2'] = train_test['impression_time_in_seconds']-train_test['prev_impression_time_2']
#train_test['next_impression_time_2'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(-2)
#train_test['time_gap_between_next_2'] = train_test['impression_time_in_seconds']-train_test['next_impression_time_2']
train_test.drop(columns=['prev_impression_time_2'],inplace=True)

In [ ]:
train_test['prev_impression_time_3'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(3)
train_test['time_gap_between_prev_3'] = train_test['impression_time_in_seconds']-train_test['prev_impression_time_3']
#train_test['next_impression_time_3'] = train_test.groupby('user_id')['impression_time_in_seconds'].shift(-3)
#train_test['time_gap_between_next_3'] = train_test['impression_time_in_seconds']-train_test['next_impression_time_3']
train_test.drop(columns=['prev_impression_time_3'],inplace=True)

In [ ]:
_ = train_test.groupby(['user_id'])['app_code'].nunique().reset_index()
_.columns=['user_id','userId_appCode_unique']
train_test = pd.merge(train_test,_,how='left',on='user_id')
_ = train_test.groupby(['user_id','app_code']).size().reset_index()
_.columns=['user_id','app_code','userId_appCode_size']
train_test = pd.merge(train_test,_,how='left',on=['user_id','app_code'])
del _
gc.collect()


In [ ]:
## os_version
_ = train_test.groupby(['user_id'])['os_version'].nunique().reset_index()
_.columns=['user_id','userId_osVersion_unique']
train_test = pd.merge(train_test,_,how='left',on='user_id')
_ = train_test.groupby(['user_id','os_version']).size().reset_index()
_.columns=['user_id','os_version','userId_osVersion_size']
train_test = pd.merge(train_test,_,how='left',on=['user_id','os_version'])
del _
gc.collect()

In [ ]:
## is_4G
_ = train_test.groupby(['user_id'])['is_4G'].nunique().reset_index()
_.columns=['user_id','userId_is4G_unique']
train_test = pd.merge(train_test,_,how='left',on='user_id')
_ = train_test.groupby(['user_id','is_4G']).size().reset_index()
_.columns=['user_id','is_4G','userId_is4G_size']
train_test = pd.merge(train_test,_,how='left',on=['user_id','is_4G'])
del _
gc.collect()

In [ ]:
mask  = train_test['userId_appCode_size']==train_test['user_id_count']
train_test.ix[mask,'userId_appCode_size']=-1
mask1 = train_test['userId_osVersion_size']==train_test['user_id_count']
train_test.ix[mask1,'userId_osVersion_size']=-1
mask2 = train_test['userId_is4G_size']==train_test['user_id_count']
train_test.ix[mask2,'userId_is4G_size']=-1

In [ ]:
## impression_id
_ = train_test.groupby(['user_id'])['impression_id'].nunique().reset_index()
_.columns=['user_id','user_impression_nunique']
train_test = pd.merge(train_test,_,how='left',on=['user_id'])
train_test['user_impression_ratio'] = train_test['user_impression_nunique']/train_test['user_id_count']

In [ ]:
_ = train_test.groupby(['user_id'])['impression_time_in_seconds'].first().reset_index()
_.columns = ['user_id','first_time_user_see_impression']
train_test = pd.merge(train_test,_,how='left',on=['user_id'])

#_ = train_test.groupby(['user_id'])['impression_time_in_seconds'].last().reset_index()
#_.columns = ['user_id','last_time_user_see_impression']
#train_test = pd.merge(train_test,_,how='left',on=['user_id'])
#del _
#gc.collect()

train_test['diff_current_first_impression_time'] = train_test['impression_time_in_seconds']-train_test['first_time_user_see_impression']

In [ ]:
train_test.shape

In [ ]:
train_test.columns

In [ ]:
view_log = pd.merge(view_log,item_data,on='item_id',how='left')
del item_data
gc.collect()

In [ ]:
view_log.head(100)

In [ ]:

def label_encode_df(dataframe,cols):
    for col in cols:
        le=preprocessing.LabelEncoder()
        dataframe[str(col)+'_encode']=le.fit_transform(dataframe[col].astype(str))

In [ ]:
label_encode_df(train_test,['os_version'])
label_encode_df(view_log,['device_type'])

In [ ]:
date_cols = ['server_time']
for date_col in date_cols:
    view_log[date_col + "_in_seconds"] = (view_log[date_col] - dt.datetime(2000,1,1)).dt.total_seconds()
    view_log[date_col + "_month"] = view_log[date_col].dt.month
    #train_df[date_col + "_year"] = train_df[date_col].dt.year
    #test_df[date_col + "_year"] = test_df[date_col].dt.year
    view_log[date_col + "_week"] = view_log[date_col].dt.week
    view_log[date_col + "_weekDay"] = view_log[date_col].dt.weekday

android = 0<br>
andoid+iphone=20<br>
android+web=1<br>
iphone=2<br>

In [ ]:
#view_log['device_encode'].unique()

In [ ]:
_ = view_log.groupby('user_id')['device_type'].nunique().reset_index()
_.columns= ['user_id','device_type_nunique']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')


_ = view_log.groupby('user_id')['device_encode'].first().reset_index()
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _
gc.collect()

In [ ]:
train_test['device_encode'].unique()

In [ ]:
user_view = view_log['user_id'].unique()
user_train = train_test['user_id'].unique()
len([i for i in user_train if i not in user_view ])

In [ ]:
view_log[view_log['user_id']==19223]

In [ ]:
_ = view_log.groupby('user_id')['session_id'].nunique().reset_index()
_.columns= ['user_id','session_id_nunique']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _
gc.collect()


In [ ]:
_ = view_log.groupby('user_id')['item_id'].nunique().reset_index()
_.columns= ['user_id','item_id_nunique']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _
gc.collect()

In [ ]:
_ = view_log.groupby(['user_id']).size().reset_index()
_.columns = ['user_id','no_of_time_user_connected']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _ 
gc.collect()

In [ ]:
_ = view_log.groupby('user_id')['server_time_in_seconds'].mean().reset_index()
_.columns = ['user_id','user_server_time_in_second_mean']
view_log = pd.merge(view_log,_,on='user_id',how='left')
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _ 
gc.collect()

In [ ]:
train_test['user_session_ratio'] = train_test['session_id_nunique']/train_test['no_of_time_user_connected']
train_test['user_item_ratio'] = train_test['item_id_nunique']/train_test['no_of_time_user_connected']

view_log['user_session_ratio'] = view_log['session_id_nunique']/view_log['no_of_time_user_connected']
view_log['user_item_ratio'] = view_log['item_id_nunique']/view_log['no_of_time_user_connected']

In [ ]:
_ = view_log.groupby(['user_id','session_id'])['server_time_in_seconds'].first().reset_index()
_.columns = ['user_id','session_id','first']
view_log = pd.merge(view_log,_,how='left',on=['user_id','session_id'])

_ = view_log.groupby(['user_id','session_id'])['server_time_in_seconds'].last().reset_index()
_.columns = ['user_id','session_id','last']
view_log = pd.merge(view_log,_,how='left',on=['user_id','session_id'])
view_log['overall_session_time'] = view_log['last']-view_log['first']
del _ 
gc.collect()


_ = view_log.groupby(['user_id'])['overall_session_time'].sum().reset_index()
_.columns = ['user_id','overall_time_sum']
view_log = pd.merge(view_log,_,how='left',on='user_id')
train_test = pd.merge(train_test,_,on='user_id',how='left')

del _
gc.collect()

In [ ]:
view_log.drop(columns=['first','last','overall_session_time'],inplace=True)

In [ ]:
_ = view_log.groupby(["user_id"])["item_price"].agg(["min", "max", "mean", "std"]).reset_index()
_.columns   = ['user_id','user_item_price_min','user_item_price_max','user_item_price_mean','user_item_price_std']
train_test = pd.merge(train_test,_,on='user_id',how='left')
view_log = pd.merge(view_log,_,how='left',on='user_id')
del _
gc.collect()

In [ ]:
_ = view_log.groupby(['user_id'])['server_time_in_seconds'].first().reset_index()
_.columns = ['user_id','first_time_user_connected_on_server']
view_log = pd.merge(view_log,_,how='left',on=['user_id'])
train_test = pd.merge(train_test,_,on='user_id',how='left')


_ = view_log.groupby(['user_id'])['server_time_in_seconds'].last().reset_index()
_.columns = ['user_id','last_time_user_connected_on_server']
view_log = pd.merge(view_log,_,how='left',on=['user_id'])
train_test = pd.merge(train_test,_,on='user_id',how='left')
del _
gc.collect()

train_test['diff_last_first_Time_user_connected_on_server'] = train_test['last_time_user_connected_on_server']-train_test['first_time_user_connected_on_server']

In [ ]:
train_test.head()

In [ ]:
#train_test['diff_user_impression_session_time_in_second_mean'] = train_test['user_impression_time_in_second_mean']-train_test['user_server_time_in_second_mean']

In [ ]:
view_log.head(100)

In [ ]:
train_test.head()

In [ ]:
train_test['user_app_price_rank'] = train_test.groupby(['user_id','app_code'])['user_item_price_mean'].rank()

In [ ]:
train_test.columns

In [ ]:
train_test.shape

In [ ]:
_ = pd.pivot_table(train_test, index="user_id", columns="impression_time_weekDay", values="impression_id", aggfunc="count", fill_value=0).reset_index()
train_test = pd.merge(train_test, _, on="user_id", how="left")

_ = pd.pivot_table(train_test, index="user_id", columns="impression_time_month", values="impression_id", aggfunc="count", fill_value=0).reset_index()
train_test = pd.merge(train_test, _, on="user_id", how="left")

_ = pd.pivot_table(train_test, index="user_id", columns="impression_time_hour", values="impression_id", aggfunc="count", fill_value=0).reset_index()
train_test = pd.merge(train_test, _, on="user_id", how="left")

_ = pd.pivot_table(train_test, index="user_id", columns="impression_time_week", values="impression_id", aggfunc="count", fill_value=0).reset_index()
train_test = pd.merge(train_test, _, on="user_id", how="left")


#_ = pd.pivot_table(train_test, index="user_id", columns="app_code", values="impression_id", aggfunc="count", fill_value=0).reset_index()
#train_test = pd.merge(train_test, _, on="user_id", how="left")

del _
gc.collect()


In [ ]:
_ = train_test.groupby(['user_id'])['app_code'].shift(1)
train_test['prev_app_code_diff'] = (train_test['app_code'] == _).astype(int)
#_ = train_test.groupby(['user_id'])['app_code'].shift(-1)
#train_test['next_app_code_diff'] = (train_test['app_code'] == _).astype(int)


In [ ]:
#train_test['device_encode'].fillna(3,inplace=True)
#train_test['device_encode'] = train_test['device_encode'].astype(np.int8)

In [ ]:
train_test.head()

In [ ]:
train_test.shape

In [ ]:
train_test = train_test.sort_values(by=['impression_time','user_id'])

In [ ]:
#train_test['user_impressionTime_diff_in_seconds']  = train_test.groupby(['user_id'])['impression_time_in_seconds'].diff()

In [ ]:
gc.collect()

In [ ]:
train_test.shape

In [ ]:
view_log = reduce_mem_usage(pd.read_csv(f'{path}view_log.csv',parse_dates=['server_time',],dayfirst=True))
item_data = reduce_mem_usage(pd.read_csv(f'{path}item_data.csv'))
view_log = pd.merge(view_log,item_data,on='item_id',how='left')

In [ ]:
date_cols = ['server_time']
for date_col in date_cols:
    view_log[date_col + "_in_seconds"] = (view_log[date_col] - dt.datetime(2000,1,1)).dt.total_seconds()
    view_log[date_col + "_month"] = view_log[date_col].dt.month
    #train_df[date_col + "_year"] = train_df[date_col].dt.year
    #test_df[date_col + "_year"] = test_df[date_col].dt.year
    view_log[date_col + "_week"] = view_log[date_col].dt.week
    view_log[date_col + "_weekDay"] = view_log[date_col].dt.weekday

In [ ]:
view_log.fillna(-99,inplace=True)

In [ ]:
train_test['no_time_user_connected_in_last_6_hours']=-1
train_test['no_time_user_created_session_in_last_6_hours']=-1
train_test['no_of_unique_element_seen_in_last_6_hours']=-1
train_test['mean_price_of_itemID_in_last_6_hours']=-1
train_test['max_price_of_itemID_in_last_6_hours']=-1
train_test['sum_price_of_itemID_in_last_6_hours']=-1
train_test['std_price_of_itemID_in_last_6_hours']=-1

In [ ]:
for i in tqdm(range(train_test.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    impression_id = s[0]
    impression_time = s[1]
    user_id = s[2]
    #print(impression_id,impression_time,user_id)
    s_prev_week = pd.date_range(end=s[1], freq='6H', periods=2)
    #print(s_prev_hour[0],s[1])
    prev_one_week =s_prev_week[0]
    mask = view_log['user_id']==user_id
    _ = view_log[(mask) & (view_log['server_time']<impression_time) & (view_log['server_time']>=prev_one_week)]
    
    train_test.iloc[i,8] = _.shape[0]
    train_test.iloc[i,9] = _['session_id'].nunique()
    train_test.iloc[i,10]=_['item_id'].nunique()
    train_test.iloc[i,11]=_['item_price'].mean()
    train_test.iloc[i,12]=_['item_price'].max()
    train_test.iloc[i,13]=_['item_price'].sum()
    train_test.iloc[i,14]=_['item_price'].std()

In [ ]:
train_test['no_time_user_connected_in_last_1_hours']=-1
train_test['no_time_user_created_session_in_last_1_hour']=-1
train_test['no_of_unique_element_seen_in_last_1_hour']=-1
train_test['mean_price_of_itemID_in_last_1_hour']=-1
train_test['max_price_of_itemID_in_last_1_hour']=-1
train_test['sum_price_of_itemID_in_last_1_hour']=-1
train_test['std_price_of_itemID_in_last_1_hoour']=-1


In [ ]:
for i in tqdm(range(train_test.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    impression_id = s[0]
    impression_time = s[1]
    user_id = s[2]
    #print(impression_id,impression_time,user_id)
    s_prev_week = pd.date_range(end=s[1], freq='1H', periods=2)
    #print(s_prev_hour[0],s[1])
    prev_one_week =s_prev_week[0]
    mask = view_log['user_id']==user_id
    _ = view_log[(mask) & (view_log['server_time']<impression_time) & (view_log['server_time']>=prev_one_week)]
    
    train_test.iloc[i,8] = _.shape[0]
    train_test.iloc[i,9] = _['session_id'].nunique()
    train_test.iloc[i,10]=_['item_id'].nunique()
    train_test.iloc[i,11]=_['item_price'].mean()
    train_test.iloc[i,12]=_['item_price'].max()
    train_test.iloc[i,13]=_['item_price'].sum()
    train_test.iloc[i,14]=_['item_price'].std()

In [ ]:
train_test['no_time_user_connected_more_than_7_days']=-1
train_test['no_time_user_created_session_more_than_7_days']=-1
train_test['no_of_unique_element_seen_more_than_7_days']=-1
train_test['mean_price_of_itemID_more_than_7_days']=-1
train_test['max_price_of_itemID_more_than_7_days']=-1
train_test['sum_price_of_itemID_more_than_7_days']=-1
train_test['std_price_of_itemID_more_than_7_days']=-1

In [ ]:
for i in tqdm(range(train_test.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    impression_id = s[0]
    impression_time = s[1]
    user_id = s[2]
    #print(impression_id,impression_time,user_id)
    s_prev_week = pd.date_range(end=s[1], freq='7D', periods=2)
    #print(s_prev_hour[0],s[1])
    prev_one_week =s_prev_week[0]
    mask = view_log['user_id']==user_id
    _ = view_log[(mask) & (view_log['server_time']<impression_time)]
    
    train_test.iloc[i,8] = _.shape[0]
    train_test.iloc[i,9] = _['session_id'].nunique()
    train_test.iloc[i,10]=_['item_id'].nunique()
    train_test.iloc[i,11]=_['item_price'].mean()
    train_test.iloc[i,12]=_['item_price'].max()
    train_test.iloc[i,13]=_['item_price'].sum()
    train_test.iloc[i,14]=_['item_price'].std()

In [ ]:
train_test['no_time_user_connected_last_1_days']=-1
train_test['no_time_user_created_session_last_1_days']=-1
train_test['no_of_unique_element_seen_last_1_days']=-1
train_test['mean_price_of_itemID_last_1_days']=-1
train_test['max_price_of_itemID_last_1_days']=-1
train_test['sum_price_of_itemID_last_1_days']=-1
train_test['std_price_of_itemID_last_1_days']=-1

In [ ]:
for i in tqdm(range(train_test.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    impression_id = s[0]
    impression_time = s[1]
    user_id = s[2]
    #print(impression_id,impression_time,user_id)
    s_prev_week = pd.date_range(end=s[1], freq='1D', periods=2)
    #print(s_prev_hour[0],s[1])
    prev_one_week =s_prev_week[0]
    mask = view_log['user_id']==user_id
    _ = view_log[(mask) & (view_log['server_time']<impression_time) & (view_log['server_time']>=prev_one_week)]
    
    train_test.iloc[i,8] = _.shape[0]
    train_test.iloc[i,9] = _['session_id'].nunique()
    train_test.iloc[i,10]=_['item_id'].nunique()
    train_test.iloc[i,11]=_['item_price'].mean()
    train_test.iloc[i,12]=_['item_price'].max()
    train_test.iloc[i,13]=_['item_price'].sum()
    train_test.iloc[i,14]=_['item_price'].std()

In [ ]:
train_test['no_time_user_connected_in_last_7_days']=-1
train_test['no_time_user_created_session_in_last_7_days']=-1
train_test['no_of_unique_element_seen_in_last_7_days']=-1
train_test['mean_price_of_itemID_in_last_7_days']=-1
train_test['max_price_of_itemID_in_last_7_days']=-1
train_test['sum_price_of_itemID_in_last_7_days']=-1
train_test['std_price_of_itemID_in_last_7_days']=-1

In [ ]:
for i in tqdm(range(train_test.shape[0])):
    s = train_test.iloc[i,[0,1,2]]
    impression_id = s[0]
    impression_time = s[1]
    user_id = s[2]
    #print(impression_id,impression_time,user_id)
    s_prev_week = pd.date_range(end=s[1], freq='7D', periods=2)
    #print(s_prev_hour[0],s[1])
    prev_one_week =s_prev_week[0]
    mask = view_log['user_id']==user_id
    _ = view_log[(mask) & (view_log['server_time']<impression_time) & (view_log['server_time']>=prev_one_week)]
    
    train_test.iloc[i,8] = _.shape[0]
    train_test.iloc[i,9] = _['session_id'].nunique()
    train_test.iloc[i,10]=_['item_id'].nunique()
    train_test.iloc[i,11]=_['item_price'].mean()
    train_test.iloc[i,12]=_['item_price'].max()
    train_test.iloc[i,13]=_['item_price'].sum()
    train_test.iloc[i,14]=_['item_price'].std()

In [ ]:
train_test.shape

In [ ]:
train_test.fillna(-99,inplace=True)

In [ ]:
train_df = train_test[train_test['is_train']==1]
test_df = train_test[train_test['is_train']==0]

In [ ]:
train_df.shape

In [ ]:
list(train_test.columns)

In [ ]:
train_df.to_csv('train_df_sid.csv',index=False)
test_df.to_csv('test_df_sid.csv',index=False)

## v0_code

In [ ]:
from datetime import datetime, timedelta 
import seaborn as sns
from tqdm import tqdm_notebook
import pdb
train = pd.read_csv('../input/av-wns-hack/train_na17sgz/train.csv')
test = pd.read_csv('../input/av-wns-hack/test_aq1fgdb/test.csv')
log = pd.read_csv('../input/av-wns-hack/train_na17sgz/view_log.csv')
item = pd.read_csv('../input/av-wns-hack/train_na17sgz/item_data.csv')

train['impression_time'] = pd.to_datetime(train['impression_time'])
test['impression_time'] = pd.to_datetime(test['impression_time'])
display(train.head(),test.head())

In [ ]:
### does not assume that no two time periods for the same 

def getRRByCategory(train,test,category,dvalueDiff,trace=False,limit=True):
    print("Category:{}".format(category))
    if limit:
        LL = limit
    else:
        LL = 'NoLimit'
        dvalueDiff = 0

    train['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    test['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    testMiss = [] ### for things that are not in test
    if trace:
            pdb.set_trace()
    for _uq in tqdm_notebook(train[category].unique()):
        _i = train[train[category]==_uq].index
        _i1 = test[test[category]==_uq].index
        _tt =  train.loc[_i,'impression_time']
        _tt1 = test.loc[_i1,'impression_time']
        #print("Starting test loop")
        for time in np.unique(_tt1.values):
            if limit:
                dd = pd.to_datetime(time).day
                increment = dd-12
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff+increment)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                #print("aaaa")
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt1[_tt1==time].index
            
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore            
            except:
                meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore        
                testMiss.append(_uq)
        #print("Starting train loop")
        for time in np.unique(_tt.values):
            if limit:
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt[_tt==time].index
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
            except:
                meanRRBefore = -999
                testMiss.append(_uq)
            train.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore
        #print("-------")
    return train,test,testMiss

train,test,testMissInit = getRRByCategory(train,test,'app_code',dvalueDiff=8,trace=False,limit=False)

for values in tqdm_notebook([7,14,21]):
    train,test,testMissFinal = getRRByCategory(train,test,'app_code',dvalueDiff=values,trace=False,limit=True)

In [ ]:
train.to_csv("trainWithRRstats_v0.csv",index=False)
test.to_csv("testWithRRstats_v0.csv",index=False)

## v1_code

In [ ]:
from datetime import datetime, timedelta 
import seaborn as sns
from tqdm import tqdm_notebook
import pdb
train = pd.read_csv('../input/av-wns-hack/train_na17sgz/train.csv')
test = pd.read_csv('../input/av-wns-hack/test_aq1fgdb/test.csv')
log = pd.read_csv('../input/av-wns-hack/train_na17sgz/view_log.csv')
item = pd.read_csv('../input/av-wns-hack/train_na17sgz/item_data.csv')

train['impression_time'] = pd.to_datetime(train['impression_time'])
test['impression_time'] = pd.to_datetime(test['impression_time'])
display(train.head(),test.head())

In [ ]:
### does not assume that no two time periods for the same 

def getRRByCategory(train,test,category,dvalueDiff,trace=False,limit=True):
    print("Category:{}".format(category))
    if limit:
        LL = limit
    else:
        LL = 'NoLimit'
        dvalueDiff = 0
        
    train['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    test['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    testMiss = [] ### for things that are not in test
    if trace:
            pdb.set_trace()
    for _uq in tqdm_notebook(train[category].unique()):
        _i = train[train[category]==_uq].index
        _i1 = test[test[category]==_uq].index
        _tt =  train.loc[_i,'impression_time']
        _tt1 = test.loc[_i1,'impression_time']
        #print("Starting test loop")
        for time in np.unique(_tt1.values):
            if limit:
                dd = pd.to_datetime(time).day
                increment = dd-12
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff+increment)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                #print("aaaa")
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt1[_tt1==time].index
            
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore            
            except:
                meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore        
                testMiss.append(_uq)
        #print("Starting train loop")
        for time in np.unique(_tt.values):
            if limit:
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt[_tt==time].index
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
            except:
                meanRRBefore = -999
                testMiss.append(_uq)
            train.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore
        #print("-------")
    return train,test,testMiss

In [ ]:
train['app_code_os_version'] = train['app_code'].astype(str)+"_"+train['os_version']
test['app_code_os_version'] = test['app_code'].astype(str)+"_"+test['os_version']

train['user_id_app_code'] = train['user_id'].astype(str)+"_"+train['app_code'].astype(str)
test['user_id_app_code'] = test['user_id'].astype(str)+"_"+test['app_code'].astype(str)
for values in tqdm_notebook(['app_code_os_version','user_id_app_code']):
    train,test,testMissFinal = getRRByCategory(train,test,values,dvalueDiff=1,trace=False,limit=False)

In [ ]:
# train.to_csv("trainWithRRstatsMore.csv",index=False)
# test.to_csv("testWithRRstatsMore.csv",index=False)
train.to_csv("trainWithRRstats_v1.csv",index=False)
test.to_csv("testWithRRstats_v1.csv",index=False)

## V2_Code

In [ ]:
train = pd.read_csv('../input/av-wns-hack/train_na17sgz/train.csv')
test = pd.read_csv('../input/av-wns-hack/test_aq1fgdb/test.csv')
log = pd.read_csv('../input/av-wns-hack/train_na17sgz/view_log.csv')
item = pd.read_csv('../input/av-wns-hack/train_na17sgz/item_data.csv')

train['impression_time'] = pd.to_datetime(train['impression_time'])
test['impression_time'] = pd.to_datetime(test['impression_time'])
display(train.head(),test.head())

In [ ]:
### does not assume that no two time periods for the same 

def getRRByCategory(train,test,category,dvalueDiff,trace=False,limit=True):
    print("Category:{}".format(category))
    if limit:
        LL = limit
    else:
        LL = 'NoLimit'
        dvalueDiff = 0
        
    train['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    test['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    testMiss = [] ### for things that are not in test
    if trace:
            pdb.set_trace()
    for _uq in tqdm_notebook(train[category].unique()):
        _i = train[train[category]==_uq].index
        _i1 = test[test[category]==_uq].index
        _tt =  train.loc[_i,'impression_time']
        _tt1 = test.loc[_i1,'impression_time']
        #print("Starting test loop")
        for time in np.unique(_tt1.values):
            if limit:
                dd = pd.to_datetime(time).day
                increment = dd-12
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff+increment)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                #print("aaaa")
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt1[_tt1==time].index
            
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore            
            except:
                meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore        
                testMiss.append(_uq)
        #print("Starting train loop")
        for time in np.unique(_tt.values):
            if limit:
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt[_tt==time].index
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
            except:
                meanRRBefore = -999
                testMiss.append(_uq)
            train.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore
        #print("-------")
    return train,test,testMiss

In [ ]:
train['app_code_os_version'] = train['app_code'].astype(str)+"_"+train['os_version']
test['app_code_os_version'] = test['app_code'].astype(str)+"_"+test['os_version']

train['user_id_app_code'] = train['user_id'].astype(str)+"_"+train['app_code'].astype(str)
test['user_id_app_code'] = test['user_id'].astype(str)+"_"+test['app_code'].astype(str)
for j in ([7,14,21]):
    train,test,testMissFinal = getRRByCategory(train,test,'app_code_os_version',dvalueDiff=j,trace=False,limit=False)

In [ ]:
# train.to_csv("trainWithRRstatsMore.csv",index=False)
# test.to_csv("testWithRRstatsMore.csv",index=False)
train.to_csv("trainWithRRstats_v2.csv",index=False)
test.to_csv("testWithRRstats_v2.csv",index=False)

## log merge approx


In [ ]:
Train_Path = '../input/av-wns-hack/train_na17sgz/'
Test_Path = '../input/av-wns-hack/test_aq1fgdb/'
train = pd.read_csv(os.path.join(Train_Path,'train.csv'))
log = pd.read_csv(os.path.join(Train_Path,'view_log.csv'))
item = pd.read_csv(os.path.join(Train_Path,'item_data.csv'))
test = pd.read_csv(os.path.join(Test_Path,'test.csv'))


In [ ]:
from datetime import datetime
import datetime
def timer(tag_name):
    def timer_decorator(fn):
        def _fn(*args, **kwargs):
            s = datetime.datetime.now()
            output = fn(*args, **kwargs)
            e = datetime.datetime.now()
            print('[{}] {} completed in {}'.format(tag_name, fn.__name__, e - s))
            return output

        return _fn

    return timer_decorator

@timer("datetime conversion")
def convertDTZone(df,col='impression_time'):
    df[col] = pd.to_datetime(df[col])
    return df

train = convertDTZone(train,col='impression_time')
log = convertDTZone(log,col='server_time')
test = convertDTZone(test,col='impression_time')

In [ ]:
@timer("Generate base datetime featues")
def genDateTimeFeats(df,col='impression_time'):
    df[col+"_day"] = df[col].dt.day
    df[col+"_minute"]=df[col].dt.minute
    df[col+"_second"]=df[col].dt.second
    df[col+"_second"]=df[col].dt.month
    return df

train = genDateTimeFeats(train)
log = genDateTimeFeats(log,'server_time')
test = genDateTimeFeats(test)

In [ ]:
def getTDStats(TD):
    totsec = TD.total_seconds()
    h = totsec//3600
    m = (totsec%3600) // 60
    sec =(totsec%3600)%60 #just for reference
    return h,m
def getTDStatsD(TD):
    totsec = TD.total_seconds()
    h = totsec//3600
    m = (totsec%3600) // 60
    sec =(totsec%3600)%60 #just for reference
    return h
def getTDStatsM(TD):
    totsec = TD.total_seconds()
    h = totsec//3600
    m = (totsec%3600) // 60
    sec =(totsec%3600)%60 #just for reference
    return m

In [ ]:
from scipy import spatial
import pdb
from tqdm import tqdm_notebook,tqdm
@timer('Log present H hours b4 the adv click')
def getLogPresentIndicator(df,log,threshold = 10):
    df["LogPresent_{}_hour_before".format(threshold)] = 0
    df["Count_LogPresent_{}_hour_before".format(threshold)] = 0
    for uID,uDF in tqdm_notebook(df[['user_id','impression_time']].groupby(by='user_id')):
        #pdb.set_trace()
        temp = log.loc[log['user_id']==uID,:]
        adArr = pd.to_datetime(uDF['impression_time'])
        logArr = pd.to_datetime(temp['server_time'])
        for qpt in adArr: ### we need to check time of ad impression > h hours from time of log impression
            idxQpt = adArr[adArr==qpt].index
            diff = (qpt - logArr) ## we need diff> 0 only for now and les than someting say 1 hour
            nullTD = datetime.timedelta(0)
            try:
                cut1,m = diff[diff>nullTD],min(diff[diff>nullTD])
                hour,mins = getTDStats(m)
                H = cut1.apply(getTDStatsD)
                M =  cut1.apply(getTDStatsM)
                c = H[(H>=0) & (H < threshold)].index
                c1 = M[M>=0][c]
                print("prelim stats",cut1.shape[0],hour,mins)
                if ((cut1.shape[0] != [])  and (hour >= 0) and (hour < threshold) 
                    and (mins>=0)):
                    print("shape",cut1.shape[0])
                    df.loc[idxQpt,"LogPresent_{}_hour_before".format(threshold)] = 1
                    df.loc[idxQpt,"Count_LogPresent_{}_hour_before".format(threshold)] = c1.shape[0]
                    print("Success")
            except:
                print("diff is null vector")   
    return df

In [ ]:
for th in [1,8,12]:   ### 1 can be used for qc, all vals in test will be 0 when th = 1 
    train = getLogPresentIndicator(train,log,th*24)
    test = getLogPresentIndicator(test,log,th*24)
    gc.collect()


In [ ]:
train.to_csv('train.csv',index=False)
test.to_csv('test.csv',index=False)

In [ ]:
#Path_consol = '../input/wns-hack-v0'
Path_raw = '../input/av-wns-hack/train_na17sgz'
train = pd.read_csv('train.csv')
test = pd.read_csv('test.csv')
log = pd.read_csv(os.path.join(Path_raw,'view_log.csv'))
item = pd.read_csv(os.path.join(Path_raw,'item_data.csv'))

@timer("Left Merge")
def merge(df1,df2,onCol):
    return pd.merge(df1,df2,on=onCol,how='left')
### merge item details with log
log = merge(log,item,'item_id')
log.head()

from tqdm import tqdm_notebook
import pdb

@timer('merge from log approximately')
def getMergeFromLog(df,log):
    name = "NearestLog_"
    df.loc[:,name+"server_time"] = 0
#     df.loc[:,name+"device_type_0"] = 'Missing'
    df.loc[:,name+"user_id"] = 'Missing'  ### for QC
#     df.loc[:,name+"item_id"] = 'Missing'

    for uID,uDF in tqdm_notebook(df[['user_id','impression_time']].groupby(by='user_id')):
#         pdb.set_trace()
        temp = log.loc[log['user_id']==uID,:]
        adArr = pd.to_datetime(uDF['impression_time'])
        logArr = pd.to_datetime(temp['server_time'])
        for qpt in adArr:
            idxQpt = adArr[adArr==qpt].index
            diff = (qpt - logArr)
            diff.index = logArr.index = temp.index
            nullTD = datetime.timedelta(0)
            if len(diff[diff>nullTD]) >= 1:
                try:
                    cut1,m = diff[diff>nullTD],min(diff[diff>nullTD])
                    mIDx = (cut1[cut1==m].index)
                    potentialLogRows = temp[temp.index.isin(mIDx)]               
                    df.loc[idxQpt,name+"server_time"] = potentialLogRows['server_time'].values[0] ### has to be same as min value is same
                    df.loc[idxQpt,name+"user_id"] = potentialLogRows['user_id'].values[0]  ### for QC, should be same
                    for t in range(len(potentialLogRows['device_type'].values.astype(list))):
                        df.loc[idxQpt,name+"device_type_{}".format(t)] = potentialLogRows['device_type'].values.astype(list)[t]
                        df.loc[idxQpt,name+"item_id_{}".format(t)] = potentialLogRows['item_id'].values.astype(list)[t]
                    #print("Success")
                except:
                    cut1,m = diff[diff>nullTD],min(diff[diff>nullTD])
                    mIDx = (cut1[cut1==m].index)
                    print("earlier",mIDx)
                    print(mIDx)
                    potentialLogRows = temp[temp.index.isin(mIDx)]
                    print(potentialLogRows,"Null diff shape")
    return df

train_new = getMergeFromLog(train,log)
test_new = getMergeFromLog(test,log)

train_new.to_csv('trainWithLogMergeApprox.csv',index=False)
test_new.to_csv('testWithLogMergeApprox.csv',index=False)

## back to morecountfe part


In [ ]:
train = pd.read_csv('trainWithLogMergeApprox.csv')
test = pd.read_csv('testWithLogMergeApprox.csv')

###
target = train['is_click']
c = [i for i in train.columns if i in test.columns]
train = train[c]
test = test[c]

train.fillna(-99,inplace=True)
test.fillna(-99,inplace=True)

badIdxTr = train[train['NearestLog_server_time']=='0']['NearestLog_server_time'].index
badIdxTe = test[test['NearestLog_server_time']=='0']['NearestLog_server_time'].index
train.loc[badIdxTr,'NearestLog_server_time'] = '2018-01-01 00:00:00'
test.loc[badIdxTe,'NearestLog_server_time'] = '2018-01-01 00:00:00'

def getTimeDelta(train,test):
    trTD = (pd.to_datetime(train['impression_time']) -pd.to_datetime(train['NearestLog_server_time']))
    teTD = (pd.to_datetime(test['impression_time']) -pd.to_datetime(test['NearestLog_server_time']))
    train['TD_Days']=trTD.dt.days
    train['TD_Hours']=(trTD.dt.seconds)//3600
    train['TD_Mins']=(trTD.dt.seconds//60)%60
    test['TD_Days']=teTD.dt.days
    test['TD_Hours']=(teTD.dt.seconds)//3600
    test['TD_Mins']=(teTD.dt.seconds//60)%60
    return train,test

train,test = getTimeDelta(train,test)
def converttoDT(train,test):
    train['impression_time'] = pd.to_datetime(train['impression_time'])
    test['impression_time'] = pd.to_datetime(test['impression_time'])
    test['NearestLog_server_time'] = pd.to_datetime(test['NearestLog_server_time'])
    train['NearestLog_server_time'] = pd.to_datetime(train['NearestLog_server_time'])
    return train,test

train,test = converttoDT(train,test)

def getCountImpressionPrevDHour(minsArr,D):
    casdt = list()
    casdt.append(0)
    for i in range(len(minsArr)):
        if i == 0:
            pass
        elif i == 1:
            casdt.append(((minsArr[1]-minsArr[0])<D).astype(int))
        elif i > 1:
            i += 1
            newArr = minsArr[:i]
            cumsum = np.cumsum(newArr)
            res = cumsum[-1]-cumsum[:-1]
            res = np.sum(res<D)
            casdt.append(res)
    return pd.Series(casdt)

def getImpressionCounts(df,col):
    df[col+'_timeSinceLastImpressionMins'] = 0
    df[col+'_AvgtimeForLastImpressionMins'] = 0
    df[col+'_StdtimeForLastImpressionMins'] = 0
    df[col+'_NCountImpressions'] = 0
    for uq in tqdm(df[col].unique()):
        temp = df[df[col]==uq]['impression_time'].sort_values()
        shift = temp.shift(1)
        diff = (temp.values-temp.shift(1,fill_value=temp.values.min()).values)
        minsArr = diff.astype('timedelta64[m]') / np.timedelta64(1, 'm')
        df.loc[temp.index,col+'_timeSinceLastImpressionMins'] = np.array(minsArr)
        df.loc[temp.index,col+'_AvgtimeForLastImpressionMins'] = np.mean(minsArr)
        df.loc[temp.index,col+'_StdtimeForLastImpressionMins'] = np.std(minsArr)
        df.loc[temp.index,col+'_NCountImpressions'] = len(temp)
    return df

from tqdm import tqdm_notebook
def getCountinLastDMins(df,col,D=[60,120]):
    for d in D:
        df['{}_CountInLast_{}_MinsImpression'.format(col,d)] = 0
    for uq in tqdm(df[col].unique()):
        temp = df[df[col]==uq]['impression_time'].sort_values()
        shift = temp.shift(1)
        diff = (temp.values-temp.shift(1,fill_value=temp.values.min()).values)
        minsArr = diff.astype('timedelta64[m]') / np.timedelta64(1, 'm')
        for d in D:
            df.loc[temp.index,'{}_CountInLast_{}_MinsImpression'.format(col,d)] = getCountImpressionPrevDHour(minsArr,d).values
    return df
consol = pd.concat([train,test],0)
consol.reset_index(inplace=True,drop=True)
consol.head()

from tqdm import tqdm
D=[60,120,300,600,24*60,24*60*2,24*60*5,24*60*7,24*60*10,24*60*14,24*60*17,24*60*21,24*60*24,24*60*28,24*60*35,24*60*42,24*60*49]
for c in tqdm(['user_id','app_code','NearestLog_item_id_0']):
    consol = getImpressionCounts(consol,c)
    consol = getCountinLastDMins(consol,c,D=D)
    print(consol.tail())
    
train = consol.iloc[:train.shape[0]]
test = consol.iloc[train.shape[0]:]
train['is_click'] = target

train.to_csv('trainLogMergeImpressionTimeStats.csv',index=False)
test.to_csv('testLogMergeImpressionTimeStats.csv',index=False)


## v3 code


In [ ]:
train = pd.read_csv('trainWithLogMergeApprox.csv')
test = pd.read_csv('testWithLogMergeApprox.csv')
train['impression_time'] = pd.to_datetime(train['impression_time'])
test['impression_time'] = pd.to_datetime(test['impression_time'])
display(train.head(),test.head())

In [ ]:
### does not assume that no two time periods for the same 

def getRRByCategory(train,test,category,dvalueDiff,trace=False,limit=True):
    print("Category:{}".format(category))
    if limit:
        LL = limit
    else:
        LL = 'NoLimit'
        dvalueDiff = 0
        
    train['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    test['BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = -999
    testMiss = [] ### for things that are not in test
    if trace:
            pdb.set_trace()
    for _uq in tqdm_notebook(train[category].unique()):
        _i = train[train[category]==_uq].index
        _i1 = test[test[category]==_uq].index
        _tt =  train.loc[_i,'impression_time']
        _tt1 = test.loc[_i1,'impression_time']
        #print("Starting test loop")
        for time in np.unique(_tt1.values):
            if limit:
                dd = pd.to_datetime(time).day
                increment = dd-12
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff+increment)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                #print("aaaa")
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt1[_tt1==time].index
            
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore            
            except:
                meanRRBefore = -999
                test.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore        
                testMiss.append(_uq)
        #print("Starting train loop")
        for time in np.unique(_tt.values):
            if limit:
                diff = pd.to_datetime(time)-timedelta(days=dvalueDiff)
                IdxBool = (_tt>=diff) & (_tt<time)
            else:
                IdxBool = (_tt<time)
 
            IdxBoolffill = _tt[_tt==time].index
            try:
                sub = _tt[IdxBool]
                meanRRBefore = train.loc[sub.index,'is_click'].mean()
                if np.isnan(meanRRBefore):
                    meanRRBefore = -999
            except:
                meanRRBefore = -999
                testMiss.append(_uq)
            train.loc[IdxBoolffill,'BeforeRunRate_{}_{}_{}'.format(category,LL,dvalueDiff)] = meanRRBefore
        #print("-------")
    return train,test,testMiss
train['app_code_NearestLog_item_id_0']= train['app_code'].astype(str)+"_"+train['NearestLog_item_id_0'].astype(str)
test['app_code_NearestLog_item_id_0']= test['app_code'].astype(str)+"_"+test['NearestLog_item_id_0'].astype(str)
for values in tqdm_notebook(['NearestLog_item_id_0','app_code_NearestLog_item_id_0']):
    train,test,testMissFinal = getRRByCategory(train,test,values,dvalueDiff=1,trace=False,limit=False)
    train,test,testMissFinal = getRRByCategory(train,test,values,dvalueDiff=7,trace=False,limit=True)

In [ ]:
train.to_csv("trainWithRRstats_v3.csv",index=False)
test.to_csv("testWithRRstats_v3.csv",index=False)


## LibFM code

In [ ]:
train = pd.read_csv('trainWithLogMergeApprox.csv')
test = pd.read_csv('testWithLogMergeApprox.csv')

In [ ]:
import pandas as pd
import numpy as np
import gc
import mlcrate as mlc
import pickle as pkl
from keras.layers.normalization import BatchNormalization
from keras.models import Sequential, Model
from keras.layers import Input, Embedding, Dense, Flatten, Concatenate, Dot, Reshape, Add, Subtract
from keras import objectives
from keras import backend as K
from keras import regularizers 
from keras.optimizers import Adam
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.regularizers import l2

import numpy as np
import tensorflow as tf
import random as rn

# The below is necessary in Python 3.2.3 onwards to
# have reproducible behavior for certain hash-based operations.
# See these references for further details:
# https://docs.python.org/3.4/using/cmdline.html#envvar-PYTHONHASHSEED
# https://github.com/fchollet/keras/issues/2280#issuecomment-306959926

import os

def init_seeds(seed):
    os.environ['PYTHONHASHSEED'] = '0'

    # The below is necessary for starting Numpy generated random numbers
    # in a well-defined initial state.

    np.random.seed(seed)

    # The below is necessary for starting core Python generated random numbers
    # in a well-defined state.

    rn.seed(seed)

    # Force TensorFlow to use single thread.
    # Multiple threads are a potential source of
    # non-reproducible results.
    # For further details, see: https://stackoverflow.com/questions/42022950/which-seeds-have-to-be-set-where-to-realize-100-reproducibility-of-training-res

    session_conf = tf.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)

    from keras import backend as K

    # The below tf.set_random_seed() will make random number generation
    # in the TensorFlow backend have a well-defined initial state.
    # For further details, see: https://www.tensorflow.org/api_docs/python/tf/set_random_seed

    tf.set_random_seed(seed)

    sess = tf.Session(graph=tf.get_default_graph(), config=session_conf)
    K.set_session(sess)
    return sess

In [ ]:
v = ['NearestLog_device_type_0', 'app_code','NearestLog_item_id_0','impression_id']
data = pd.concat([train[v],test[v]],0)
data.head()

from sklearn.preprocessing import LabelEncoder

le1 = LabelEncoder()
le1.fit(data['NearestLog_device_type_0'].astype('str'))
data['NearestLog_device_type_0_le']=le1.transform(data['NearestLog_device_type_0'].astype('str'))

features = ['NearestLog_device_type_0_le', 'app_code','NearestLog_item_id_0']
f_size  = [int(data[f].max()) + 1 for f in features]
X = data.groupby(features)['impression_id'].count()
X = X.unstack().fillna(0)
X = X.stack().astype('float32')
X = np.log1p(X).reset_index()
X.columns=features + ['num']

X_train = [X[f].values for f in features]
y_train = (X[['num']].values).astype('float32')
(X.num > 0).mean()
w_train = (50 * (y_train > 0).astype('float32') + 1).ravel()
k_latent = 2
embedding_reg = 0.0002
kernel_reg = 0.1

def get_embed(x_input, x_size, k_latent):
    if x_size > 0: #category
        embed = Embedding(x_size, k_latent, input_length=1, 
                          embeddings_regularizer=l2(embedding_reg))(x_input)
        embed = Flatten()(embed)
    else:
        embed = Dense(k_latent, kernel_regularizer=l2(embedding_reg))(x_input)
    return embed

def build_model_1(X, f_size):
    dim_input = len(f_size)
    
    input_x = [Input(shape=(1,)) for i in range(dim_input)] 
     
    biases = [get_embed(x, size, 1) for (x, size) in zip(input_x, f_size)]
    
    factors = [get_embed(x, size, k_latent) for (x, size) in zip(input_x, f_size)]
    
    s = Add()(factors)
    
    diffs = [Subtract()([s, x]) for x in factors]
    
    dots = [Dot(axes=1)([d, x]) for d,x in zip(diffs, factors)]
    
    x = Concatenate()(biases + dots)
    x = BatchNormalization()(x)
    output = Dense(1, activation='relu', kernel_regularizer=l2(kernel_reg))(x)
    model = Model(inputs=input_x, outputs=[output])
    opt = Adam(clipnorm=0.5)
    model.compile(optimizer=opt, loss='mean_squared_error')
    output_f = factors + biases
    model_features = Model(inputs=input_x, outputs=output_f)
    return model, model_features
model, model_features = build_model_1(X_train, f_size)

n_epochs = 100
P = 17
try:
    del sess
except:
    pass
sess = init_seeds(0)

batch_size = 2**P
print(batch_size)
model, model_features = build_model_1(X_train, f_size)
earlystopper = EarlyStopping(patience=0, verbose=1)

model.fit(X_train,  y_train, 
          epochs=n_epochs, batch_size=batch_size, verbose=1, shuffle=True, 
          validation_data=(X_train, y_train), 
          sample_weight=w_train,
          callbacks=[earlystopper],
         )
X_pred = model_features.predict(X_train, batch_size=batch_size)
factors = X_pred[:len(features)]

biases = X_pred[len(features):2*len(features)]

for f, X_p in zip(features, factors):
    for i in range(k_latent):
        X['%s_fm_factor_%d' % (f, i)] = X_p[:,i]

for f, X_p in zip(features, biases):
    X['%s_fm_bias' % (f)] = X_p[:,0]


X['NearestLog_device_type_0_le'] = np.array(le1.inverse_transform(np.array(X['NearestLog_device_type_0_le'])))
X.to_csv('FM_feats.csv',index=False)


In [ ]:
train_0 = pd.read_csv('trainWithRRstats_v0.csv')
test_0 = pd.read_csv('testWithRRstats_v0.csv')

train_1 = pd.read_csv('trainWithRRstats_v1.csv')
test_1 = pd.read_csv('testWithRRstats_v1.csv')

cols_1=['app_code_os_version','BeforeRunRate_app_code_os_version_NoLimit_0',
       'BeforeRunRate_user_id_app_code_NoLimit_0']
train_consol = pd.concat([train_0,train_1[cols_1]],1)
test_consol = pd.concat([test_0,test_1[cols_1]],1)
del train_1,test_1,train_0,test_0

train_2 = pd.read_csv('trainWithRRstats_v2.csv')
test_2 = pd.read_csv('testWithRRstats_v2.csv')
cols_2 = ['user_id_app_code','BeforeRunRate_app_code_os_version_NoLimit_0']
train_consol = pd.concat([train_consol,train_2[cols_2]],1)
test_consol = pd.concat([test_consol,test_2[cols_2]],1)
del train_2,test_2

train_3 = pd.read_csv('trainWithRRstats_v3.csv')
test_3 = pd.read_csv('testWithRRstats_v3.csv')
keep = [i for i in test_3.columns if i in train_3.columns]
train_3 = train_3[keep]
cols_3=[i for i in train_3.columns if i not in train_consol.columns]
train_consol = pd.concat([train_consol,train_3[cols_3]],1)
test_consol = pd.concat([test_consol,test_3[cols_3]],1)
del train_3,test_3

train_consol['impression_weekday']=pd.to_datetime(train_consol['impression_time']).dt.weekday
test_consol['impression_weekday']=pd.to_datetime(test_consol['impression_time']).dt.weekday
train_consol.loc[train_consol['NearestLog_server_time']=='0','NearestLog_server_time'] = pd.to_datetime('2018-01-01 00:00:00')
test_consol.loc[test_consol['NearestLog_server_time']=='0','NearestLog_server_time'] = pd.to_datetime('2018-01-01 00:00:00')
train_consol['NearestLog_server_time_weekday']=pd.to_datetime(train_consol['NearestLog_server_time']).dt.weekday
test_consol['NearestLog_server_time_weekday']=pd.to_datetime(test_consol['NearestLog_server_time']).dt.weekday
train_consol['NearestLog_server_time_day']=pd.to_datetime(train_consol['NearestLog_server_time']).dt.day
test_consol['NearestLog_server_time_day']=pd.to_datetime(test_consol['NearestLog_server_time']).dt.day
train_consol['NearestLog_server_time_month']=pd.to_datetime(train_consol['NearestLog_server_time']).dt.month
test_consol['NearestLog_server_time_month']=pd.to_datetime(test_consol['NearestLog_server_time']).dt.month
train_consol['NearestLog_server_time_hour']=pd.to_datetime(train_consol['NearestLog_server_time']).dt.hour
test_consol['NearestLog_server_time_hour']=pd.to_datetime(test_consol['NearestLog_server_time']).dt.hour
train_4 = pd.read_csv('trainLogMergeImpressionTimeStats.csv')
test_4 = pd.read_csv('testLogMergeImpressionTimeStats.csv')
c = [i for i in train_4.columns if i not in train_consol.columns]

train_consol = pd.concat([train_consol,train_4[c]],1)
test_consol = pd.concat([test_consol,test_4[c]],1)

#ls '../input/av-wns-hack/train_na17sgz/'
log = pd.read_csv('../input/av-wns-hack/train_na17sgz/view_log.csv')
item = pd.read_csv('../input/av-wns-hack/train_na17sgz/item_data.csv')

train_consol.replace(-999.0, 0,inplace=True)
test_consol.replace(-999.0, 0,inplace=True)

train_consol.fillna(-99,inplace=True)
test_consol.fillna(-99,inplace=True)

itemIdCols = ['NearestLog_item_id_0','NearestLog_item_id_1','NearestLog_item_id_2','NearestLog_item_id_3',
             'NearestLog_item_id_4','NearestLog_item_id_5','NearestLog_item_id_6','NearestLog_item_id_7']

from tqdm import tqdm_notebook
import pdb
neigh = 0
for c in tqdm_notebook(itemIdCols):
    Ttemp = pd.merge(train_consol[c],item,left_on=c,right_on='item_id',how='left')
    Ttemp.drop([c,'item_id'],1,inplace=True)
    columnName = Ttemp.columns
    newNames = [i+"_"+str(neigh) for i in columnName]
    dc = dict(zip(columnName,newNames))
    Ttemp.rename(columns=dc,inplace=True)
    Tetemp = pd.merge(test_consol[c],item,left_on=c,right_on='item_id',how='left')
    Tetemp.drop([c,'item_id'],1,inplace=True)
    Tetemp.rename(columns=dc,inplace=True)
    train_consol = pd.concat([train_consol,Ttemp],1)
    test_consol = pd.concat([test_consol,Tetemp],1)
    neigh += 1

train_consol.fillna(-99,inplace=True)
test_consol.fillna(-99,inplace=True)

target = train_consol['is_click']
train_consol.drop('is_click',1,inplace=True)
v = ['os_version', 'NearestLog_device_type_0', 'app_code','NearestLog_item_id_0','impression_id']
consol = pd.concat([train_consol,test_consol],0)

def newCountStats(df,v1,train,test,cname):  ### df = consol
    X = df.groupby(v1)['impression_id'].count()
    X = X.unstack().fillna(0)
    X = X.stack().astype('float32')
    X = np.log1p(X).reset_index()
    X.columns=v1 + [cname]
    train = pd.merge(train,X,on=v1,how='left')
    test = pd.merge(test,X,on=v1,how='left')
    train.fillna(0,inplace=True)
    test.fillna(0,inplace=True)
    return train,test
v1 = ['os_version', 'NearestLog_device_type_0', 'app_code','NearestLog_item_id_0']
v2 = ['user_id', 'app_code']
v3 = ['NearestLog_device_type_0', 'app_code']
v4 = ['NearestLog_device_type_0', 'app_code','NearestLog_device_type_1']
v5 = ['NearestLog_device_type_0', 'user_id','app_code']

count  = 1
for vs in tqdm_notebook([v1,v2,v3,v4,v5]):
    name = "num_{}".format(count) 
    train_consol,test_consol = newCountStats(consol,vs,train_consol,test_consol,name)
    count += 1
import gc
gc.collect()

def do_countuniq( df, group_cols, counted, agg_type='uint8', show_max=False, show_agg=True ):
    agg_name= '{}_by_{}_countuniq'.format(('_'.join(group_cols)),(counted))  
    if show_agg:
        print( "\nCounting unqiue ", counted, " by ", group_cols ,  '... and saved in', agg_name )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].nunique().reset_index().rename(columns={counted:agg_name})
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return( df )
### Below a function is written to extract cumulative count feature  from different cols    
def do_cumcount( df, group_cols, counted,agg_type='uint16', show_max=False, show_agg=True ):
    agg_name= '{}_by_{}_cumcount'.format(('_'.join(group_cols)),(counted)) 
    if show_agg:
        print( "\nCumulative count by ", group_cols , '... and saved in', agg_name  )
    gp = df[group_cols+[counted]].groupby(group_cols)[counted].cumcount()
    df[agg_name]=gp.values
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return( df )
## Below a function is written to extract count feature by aggregating different cols
def do_count( df, group_cols, agg_type='uint16', show_max=False, show_agg=True ):
    agg_name='{}count'.format('_'.join(group_cols))  
    if show_agg:
        print( "\nAggregating by ", group_cols ,  '... and saved in', agg_name )
    gp = df[group_cols][group_cols].groupby(group_cols).size().rename(agg_name).to_frame().reset_index()
    df = df.merge(gp, on=group_cols, how='left')
    del gp
    if show_max:
        print( agg_name + " max value = ", df[agg_name].max() )
    df[agg_name] = df[agg_name].astype(agg_type)
    gc.collect()
    return( df )
train_consol['impression_time_hour']=pd.to_datetime(train_consol['impression_time']).dt.hour
test_consol['impression_time_hour']=pd.to_datetime(test_consol['impression_time']).dt.hour
train_consol['impression_time_month']=pd.to_datetime(train_consol['impression_time']).dt.month
test_consol['impression_time_month']=pd.to_datetime(test_consol['impression_time']).dt.month

train_df = train_consol.copy()
cols = [i for i in train_df.columns if i in test_consol.columns]
len_train = len(train_df)
train_df=train_df[cols].append(test_consol[cols])
train_df.reset_index(drop=True,inplace=True)
#del test

gc.collect()

train_df = do_countuniq( train_df, ['user_id'], 'os_version' ); gc.collect()
train_df = do_countuniq( train_df, ['user_id', 'NearestLog_device_type_0', 'os_version'], 'app_code'); gc.collect()
train_df = do_countuniq( train_df, ['user_id', 'impression_time_day'], 'impression_time_hour' ); gc.collect()
train_df = do_countuniq( train_df, ['user_id'], 'app_code'); gc.collect()
train_df = do_countuniq( train_df, ['user_id', 'app_code'], 'os_version'); gc.collect()
train_df = do_countuniq( train_df, ['user_id'], 'NearestLog_device_type_0'); gc.collect()
train_df = do_countuniq( train_df, ['app_code'], 'os_version'); gc.collect()

train_df = do_cumcount( train_df, ['user_id'], 'os_version'); gc.collect()
train_df = do_cumcount( train_df, ['user_id', 'NearestLog_device_type_0', 'os_version'], 'app_code'); gc.collect()
train_df = do_count( train_df, ['user_id', 'impression_time_day', 'impression_time_hour'] ); gc.collect()
train_df = do_count( train_df, ['user_id', 'app_code']); gc.collect()
train_df = do_count( train_df, ['user_id', 'app_code', 'os_version']); gc.collect()

# FMFeats = pd.read_csv('FM_feats.csv')

# train_df = pd.merge(train_df,FMFeats,left_on=['NearestLog_device_type_0','app_code','NearestLog_item_id_0']
#                     ,right_on=['NearestLog_device_type_0_le','app_code','NearestLog_item_id_0'],how='left')

# train_df.drop('NearestLog_device_type_0_le',1,inplace=True)

from sklearn import preprocessing
cats = ['app_code','os_version','is_4G','app_code_NearestLog_item_id_0','app_code_os_version',
        'user_id_app_code','NearestLog_device_type_0','NearestLog_device_type_1', 
        'NearestLog_device_type_2',
         'NearestLog_device_type_3','NearestLog_device_type_4', 'NearestLog_device_type_5',
         'NearestLog_device_type_6', 'NearestLog_device_type_7',
       'NearestLog_item_id_0', 'NearestLog_item_id_1', 'NearestLog_item_id_2','NearestLog_item_id_3',
        'NearestLog_item_id_4','NearestLog_item_id_5','NearestLog_item_id_6', 'NearestLog_item_id_7',        
        'impression_weekday','NearestLog_server_time_weekday']
categoryID = [i for i in train_consol.columns if 'category_' in i]
pdtID = [i for i in train_consol.columns if 'product_type_' in i]

cats = cats+categoryID+pdtID
for j in tqdm_notebook(cats):
    print(j)
    le = preprocessing.LabelEncoder()
    train_df[j] = train_df[j].astype('str')
    train_df[j] = le.fit_transform(train_df[j])
    
train_consol = train_df.iloc[:train_consol.shape[0],:]
test_consol = train_df.iloc[train_consol.shape[0]:,:]

train_consol.fillna(-99,inplace=True)
test_consol.fillna(-99,inplace=True)

drop = ['impression_time','user_id','NearestLog_server_time','NearestLog_user_id']
train_consol['is_click'] = target
train_consol.drop(drop,1,inplace=True)
test_consol.drop(drop,1,inplace=True)

train_consol.to_csv('consolTrain.csv',index=False)
test_consol.to_csv('consolTest.csv',index=False)

In [ ]:
train = pd.read_csv('consolTrain.csv')
test = pd.read_csv('consolTest.csv')

print(train.shape,test.shape)

In [ ]:
contingencyCandidates = [('app_code','NearestLog_item_id_0'),('app_code','NearestLog_item_id_1'),
                         ('app_code','NearestLog_device_type_0')
                        ,('app_code','NearestLog_device_type_1'),('app_code','impression_weekday'),
('app_code','product_type_0'),('app_code','product_type_1'),('app_code','category_1_1'),
('app_code','category_2_1'),('app_code','category_3_1'),('app_code','category_1_2'),
('app_code','category_2_2'),('app_code','category_3_2'),('app_code','category_3_0'),
('product_type_0','category_1_1'),('product_type_1','category_1_1'),
('product_type_0','category_1_2'),('product_type_1','category_1_2'),('product_type_1','category_3_1'),
 ('product_type_0','category_3_1'),('app_code','NearestLog_server_time_weekday')]

cols = [i for i in train.columns if i in test.columns]
Y = train['is_click']
consol = pd.concat([train[cols],test[cols]],0)
consol.reset_index(drop=True,inplace=True)

def getNMFFeatures(df,contingencyCandidates):
    from sklearn.decomposition import NMF
    from tqdm import tqdm_notebook
    for cc in tqdm_notebook(contingencyCandidates):
        l,r = cc
        print(l,r)
        cMat = pd.crosstab(df[l],df[r])
        model = NMF(n_components=20, init='random', alpha=0.1,l1_ratio=0.1,random_state=0)
        W = model.fit_transform(cMat)
        H = model.components_
        colDF = pd.concat([pd.DataFrame(cMat.columns),pd.DataFrame(H.T)],1)
        prefix = l+"_"+r+"_NMF"
        newNames = [prefix+str(i)+"_1" for i in range(colDF.shape[1]-1)]
        oldNames = colDF.columns[1:]
        mapper = dict(zip(oldNames,newNames))
        colDF.rename(columns=mapper,inplace=True)
        rowDF = pd.concat([pd.DataFrame(cMat.index),pd.DataFrame(W)],1)
        newNames = [prefix+str(i)+'_2' for i in range(rowDF.shape[1]-1)]
        oldNames = rowDF.columns[1:]
        mapper = dict(zip(oldNames,newNames))
        rowDF.rename(columns=mapper,inplace=True)  
        df = pd.merge(df,colDF,on=colDF.columns[0],how='left')
        df = pd.merge(df,rowDF,on=rowDF.columns[0],how='left')
    return df
consol = getNMFFeatures(consol,contingencyCandidates)


train = consol.iloc[:train.shape[0],:]
train['is_click'] = Y
test = consol.iloc[train.shape[0]:,:]
train.to_csv('consoltrainNMFFeats.csv',index=False)
test.to_csv('consoltestNMFFeats.csv',index=False)

In [ ]:
train_df_sid = pd.read_pickle(f'train_df_sid.pkl')
test_df_sid = pd.read_pickle(f'test_df_sid.pkl')

In [ ]:
train_df = pd.read_csv(f'consoltrainNMFFeats.csv')
test_df = pd.read_csv(f'consoltestNMFFeats.csv')

In [ ]:
train_df_sid.drop(columns=['is_train','is_click','app_code','os_version','is_4G'],inplace=True)
test_df_sid.drop(columns=['is_train','is_click','app_code','os_version','is_4G'],inplace=True)

In [ ]:
train_df = pd.merge(train_df,train_df_sid,how='left',on='impression_id')
test_df = pd.merge(test_df,test_df_sid,how='left',on='impression_id')

In [ ]:
train_df.shape

In [ ]:
test_df.shape

In [ ]:
train_df.to_pickle('train_df_merge.pkl')
test_df.to_pickle('test_df_merge.pkl')

In [ ]:
train_df = pd.read_pickle(f'train_df_merge.pkl')
test_df = pd.read_pickle(f'test_df_merge.pkl')

In [ ]:
train_df['is_train']=1
test_df['is_train']=0

In [ ]:
train_test = pd.concat([train_df,test_df],axis=0,sort=False).reset_index(drop=True)

In [ ]:
_ = train_test.groupby('app_code_os_version')['item_price_1'].mean().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice1_mean']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')         
_ = train_test.groupby('app_code_os_version')['item_price_2'].mean().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice2_mean']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_3'].mean().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice3_mean']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_4'].mean().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice4_mean']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_5'].mean().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice5_mean']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_6'].mean().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice6_mean']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_7'].mean().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice7_mean']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_1'].sum().reset_index()
_ = _[_['item_price_1']!=-99]


_.columns = ['app_code_os_version','app_code_os_version_itemPrice1_sum']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_2'].sum().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice2_sum']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_3'].sum().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice3_sum']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_4'].sum().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice4_sum']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_5'].sum().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice5_sum']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_6'].sum().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice6_sum']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_7'].sum().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice7_sum']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')






_ = train_test.groupby('app_code_os_version')['item_price_1'].max().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice1_max']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_2'].max().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice2_max']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_3'].max().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice3_max']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_4'].max().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice4_max']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_5'].max().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice5_max']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_6'].max().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice6_max']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_7'].max().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice7_max']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')



_ = train_test.groupby('app_code_os_version')['item_price_1'].min().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice1_min']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_2'].min().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice2_min']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_3'].min().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice3_min']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_4'].min().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice4_min']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_5'].min().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice5_min']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_6'].min().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice6_min']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')
_ = train_test.groupby('app_code_os_version')['item_price_7'].min().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['app_code_os_version','app_code_os_version_itemPrice7_min']
train_test = pd.merge(train_test,_,on='app_code_os_version',how='left')

In [ ]:
_ = train_test.groupby('app_code')['item_price_1'].mean().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['app_code','app_itemPrice1_mean']
train_test = pd.merge(train_test,_,on='app_code',how='left')         
_ = train_test.groupby('app_code')['item_price_2'].mean().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['app_code','app_itemPrice2_mean']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_3'].mean().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['app_code','app_itemPrice3_mean']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_4'].mean().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['app_code','app_itemPrice4_mean']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_5'].mean().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['app_code','app_itemPrice5_mean']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_6'].mean().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['app_code','app_itemPrice6_mean']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_7'].mean().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['app_code','app_itemPrice7_mean']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_1'].sum().reset_index()
_ = _[_['item_price_1']!=-99]


_.columns = ['app_code','app_itemPrice1_sum']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_2'].sum().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['app_code','app_itemPrice2_sum']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_3'].sum().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['app_code','app_itemPrice3_sum']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_4'].sum().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['app_code','app_itemPrice4_sum']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_5'].sum().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['app_code','app_itemPrice5_sum']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_6'].sum().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['app_code','app_itemPrice6_sum']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_7'].sum().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['app_code','app_itemPrice7_sum']
train_test = pd.merge(train_test,_,on='app_code',how='left')






_ = train_test.groupby('app_code')['item_price_1'].max().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['app_code','app_itemPrice1_max']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_2'].max().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['app_code','app_itemPrice2_max']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_3'].max().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['app_code','app_itemPrice3_max']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_4'].max().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['app_code','app_itemPrice4_max']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_5'].max().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['app_code','app_itemPrice5_max']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_6'].max().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['app_code','app_itemPrice6_max']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_7'].max().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['app_code','app_itemPrice7_max']
train_test = pd.merge(train_test,_,on='app_code',how='left')



_ = train_test.groupby('app_code')['item_price_1'].min().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['app_code','app_itemPrice1_min']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_2'].min().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['app_code','app_itemPrice2_min']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_3'].min().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['app_code','app_itemPrice3_min']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_4'].min().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['app_code','app_itemPrice4_min']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_5'].min().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['app_code','app_itemPrice5_min']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_6'].min().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['app_code','app_itemPrice6_min']
train_test = pd.merge(train_test,_,on='app_code',how='left')
_ = train_test.groupby('app_code')['item_price_7'].min().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['app_code','app_itemPrice7_min']
train_test = pd.merge(train_test,_,on='app_code',how='left')

In [ ]:
_ = train_test.groupby('is_4G')['item_price_1'].mean().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['is_4G','is_4G_itemPrice1_mean']
train_test = pd.merge(train_test,_,on='is_4G',how='left')         
_ = train_test.groupby('is_4G')['item_price_2'].mean().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['is_4G','is_4G_itemPrice2_mean']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_3'].mean().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['is_4G','is_4G_itemPrice3_mean']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_4'].mean().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['is_4G','is_4G_itemPrice4_mean']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_5'].mean().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['is_4G','is_4G_itemPrice5_mean']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_6'].mean().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['is_4G','is_4G_itemPrice6_mean']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_7'].mean().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['is_4G','is_4G_itemPrice7_mean']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_1'].sum().reset_index()
_ = _[_['item_price_1']!=-99]


_.columns = ['is_4G','is_4G_itemPrice1_sum']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_2'].sum().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['is_4G','is_4G_itemPrice2_sum']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_3'].sum().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['is_4G','is_4G_itemPrice3_sum']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_4'].sum().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['is_4G','is_4G_itemPrice4_sum']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_5'].sum().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['is_4G','is_4G_itemPrice5_sum']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_6'].sum().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['is_4G','is_4G_itemPrice6_sum']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_7'].sum().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['is_4G','is_4G_itemPrice7_sum']
train_test = pd.merge(train_test,_,on='is_4G',how='left')






_ = train_test.groupby('is_4G')['item_price_1'].max().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['is_4G','is_4G_itemPrice1_max']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_2'].max().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['is_4G','is_4G_itemPrice2_max']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_3'].max().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['is_4G','is_4G_itemPrice3_max']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_4'].max().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['is_4G','is_4G_itemPrice4_max']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_5'].max().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['is_4G','is_4G_itemPrice5_max']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_6'].max().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['is_4G','is_4G_itemPrice6_max']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_7'].max().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['is_4G','is_4G_itemPrice7_max']
train_test = pd.merge(train_test,_,on='is_4G',how='left')



_ = train_test.groupby('is_4G')['item_price_1'].min().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['is_4G','is_4G_itemPrice1_min']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_2'].min().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['is_4G','is_4G_itemPrice2_min']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_3'].min().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['is_4G','is_4G_itemPrice3_min']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_4'].min().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['is_4G','is_4G_itemPrice4_min']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_5'].min().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['is_4G','is_4G_itemPrice5_min']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_6'].min().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['is_4G','is_4G_itemPrice6_min']
train_test = pd.merge(train_test,_,on='is_4G',how='left')
_ = train_test.groupby('is_4G')['item_price_7'].min().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['is_4G','is_4G_itemPrice7_min']
train_test = pd.merge(train_test,_,on='is_4G',how='left')

In [ ]:
_ = train_test.groupby('os_version')['item_price_1'].mean().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['os_version','os_version_itemPrice1_mean']
train_test = pd.merge(train_test,_,on='os_version',how='left')         
_ = train_test.groupby('os_version')['item_price_2'].mean().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['os_version','os_version_itemPrice2_mean']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_3'].mean().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['os_version','os_version_itemPrice3_mean']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_4'].mean().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['os_version','os_version_itemPrice4_mean']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_5'].mean().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['os_version','os_version_itemPrice5_mean']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_6'].mean().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['os_version','os_version_itemPrice6_mean']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_7'].mean().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['os_version','os_version_itemPrice7_mean']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_1'].sum().reset_index()
_ = _[_['item_price_1']!=-99]


_.columns = ['os_version','os_version_itemPrice1_sum']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_2'].sum().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['os_version','os_version_itemPrice2_sum']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_3'].sum().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['os_version','os_version_itemPrice3_sum']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_4'].sum().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['os_version','os_version_itemPrice4_sum']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_5'].sum().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['os_version','os_version_itemPrice5_sum']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_6'].sum().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['os_version','os_version_itemPrice6_sum']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_7'].sum().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['os_version','os_version_itemPrice7_sum']
train_test = pd.merge(train_test,_,on='os_version',how='left')






_ = train_test.groupby('os_version')['item_price_1'].max().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['os_version','os_version_itemPrice1_max']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_2'].max().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['os_version','os_version_itemPrice2_max']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_3'].max().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['os_version','os_version_itemPrice3_max']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_4'].max().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['os_version','os_version_itemPrice4_max']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_5'].max().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['os_version','os_version_itemPrice5_max']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_6'].max().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['os_version','os_version_itemPrice6_max']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_7'].max().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['os_version','os_version_itemPrice7_max']
train_test = pd.merge(train_test,_,on='os_version',how='left')



_ = train_test.groupby('os_version')['item_price_1'].min().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['os_version','os_version_itemPrice1_min']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_2'].min().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['os_version','os_version_itemPrice2_min']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_3'].min().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['os_version','os_version_itemPrice3_min']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_4'].min().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['os_version','os_version_itemPrice4_min']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_5'].min().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['os_version','os_version_itemPrice5_min']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_6'].min().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['os_version','os_version_itemPrice6_min']
train_test = pd.merge(train_test,_,on='os_version',how='left')
_ = train_test.groupby('os_version')['item_price_7'].min().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['os_version','os_version_itemPrice7_min']
train_test = pd.merge(train_test,_,on='os_version',how='left')

In [ ]:
_ = train_test.groupby('user_id_app_code')['item_price_1'].mean().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice1_mean']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')         
_ = train_test.groupby('user_id_app_code')['item_price_2'].mean().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice2_mean']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_3'].mean().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice3_mean']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_4'].mean().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice4_mean']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_5'].mean().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice5_mean']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_6'].mean().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice6_mean']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_7'].mean().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice7_mean']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_1'].sum().reset_index()
_ = _[_['item_price_1']!=-99]


_.columns = ['user_id_app_code','user_id_app_code_itemPrice1_sum']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_2'].sum().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice2_sum']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_3'].sum().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice3_sum']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_4'].sum().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice4_sum']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_5'].sum().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice5_sum']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_6'].sum().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice6_sum']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_7'].sum().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice7_sum']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')






_ = train_test.groupby('user_id_app_code')['item_price_1'].max().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice1_max']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_2'].max().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice2_max']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_3'].max().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice3_max']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_4'].max().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice4_max']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_5'].max().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice5_max']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_6'].max().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice6_max']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_7'].max().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice7_max']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')



_ = train_test.groupby('user_id_app_code')['item_price_1'].min().reset_index()
_ = _[_['item_price_1']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice1_min']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_2'].min().reset_index()
_ = _[_['item_price_2']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice2_min']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_3'].min().reset_index()
_ = _[_['item_price_3']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice3_min']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_4'].min().reset_index()
_ = _[_['item_price_4']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice4_min']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_5'].min().reset_index()
_ = _[_['item_price_5']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice5_min']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_6'].min().reset_index()
_ = _[_['item_price_6']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice6_min']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')
_ = train_test.groupby('user_id_app_code')['item_price_7'].min().reset_index()
_ = _[_['item_price_7']!=-99]
_.columns = ['user_id_app_code','user_id_app_code_itemPrice7_min']
train_test = pd.merge(train_test,_,on='user_id_app_code',how='left')

In [ ]:
del _
gc.collect()

In [ ]:
train_df = train_test[train_test['is_train']==1]
test_df = train_test[train_test['is_train']==0]
train_df.drop(columns='is_train',inplace=True)
test_df.drop(columns='is_train',inplace=True)

In [ ]:
train_df.to_pickle('train_df_mergeall.pkl')
test_df.to_pickle('test_df_mergeall.pkl')

In [ ]:
test_df = pd.read_pickle('train_df_mergeall.pkl')
train_df = pd.read_pickle('test_df_mergeall.pkl')


In [ ]:
train_df.columns[train_df.columns=='is_click']

In [ ]:
test_df.drop(columns='is_click',inplace=True)

In [ ]:
train_df.to_pickle('train_df_final.pkl')
test_df.to_pickle('test_df_final.pkl')

# LGBMODEL

In [ ]:
train = pd.read_pickle('train_df_final.pkl')
test = pd.read_pickle('test_df_final.pkl')

print(train.shape,test.shape)

In [ ]:
NMFCols = [i for i in train.columns if 'NMF' in i]
train.drop(NMFCols,1,inplace=True)
test.drop(NMFCols,1,inplace=True)
train.fillna(-99,inplace=True)
test.fillna(-99,inplace=True)
len(NMFCols)

In [ ]:
import gc; gc.collect()

In [ ]:
pd.set_option('display.max_rows',10)
import seaborn as sns
import gc
from tqdm import tqdm,tqdm_notebook
from datetime import datetime
import lightgbm as lgb
from sklearn.model_selection import KFold

gc.collect()

In [ ]:
def timer(tag_name):
    import datetime
    def timer_decorator(fn):
        def _fn(*args, **kwargs):
            s = datetime.datetime.now()
            output = fn(*args, **kwargs)
            e = datetime.datetime.now()
            print('[{}] {} completed in {}'.format(tag_name, fn.__name__, e - s))
            return output

        return _fn

    return timer_decorator

In [ ]:
from sklearn.metrics import auc,roc_curve,roc_auc_score
def aucROC(y_pred,dtrain):
    labels = dtrain.get_label()
    return 'AUC:',roc_auc_score(np.array(labels), np.array(y_pred)),True

In [ ]:
@timer('KFoldLGBModel')
def runLGBMWithKF(train,test,Y,param,splits,cats):
    from sklearn.model_selection import StratifiedKFold
    folds = StratifiedKFold(n_splits=splits, shuffle=True, random_state=42)
    oof_preds = np.zeros(train.shape[0])
    sub_preds = np.zeros(test.shape[0])
    feature_importance_df = pd.DataFrame()    
    for n_fold, (trn_idx, val_idx) in enumerate(folds.split(train,Y)):
        trn_x, trn_y = train.iloc[trn_idx], Y.iloc[trn_idx]
        val_x, val_y = train.iloc[val_idx], Y.iloc[val_idx]
        
        xg_train = lgb.Dataset(trn_x,
                               label=trn_y,
                               free_raw_data = False
                               )
        xg_valid = lgb.Dataset(val_x,
                               label=val_y,
                               free_raw_data = False
                               )   

        print("Fold:",n_fold+1)
        clf = lgb.train(param, xg_train, 10000, valid_sets = [xg_train,xg_valid], 
                        verbose_eval=100, early_stopping_rounds = 500,
                        feval= aucROC,  categorical_feature=cats)

        oof_preds[val_idx] = clf.predict(val_x, num_iteration=clf.best_iteration)
        sub_preds += clf.predict(test, num_iteration=clf.best_iteration)/folds.n_splits
        print('Val AUC : %.6f' % roc_auc_score(val_y,oof_preds[val_idx]))

        fold_importance_df = pd.DataFrame()
        fold_importance_df["feature"] = train.columns
        fold_importance_df["importance"] = clf.feature_importance()
        fold_importance_df["fold"] = n_fold + 1
        feature_importance_df = pd.concat([feature_importance_df, fold_importance_df], axis=0)
        gc.collect()
    print("Full AUC:",roc_auc_score(Y,oof_preds))
    return feature_importance_df,sub_preds,oof_preds,roc_auc_score(Y,oof_preds)

In [ ]:
### below give 71.9 on lb, not anymore
params = {
        'boosting_type': 'gbdt',
        'objective': 'binary',
       'metric':'auc',
        'learning_rate': 0.008,
        'is_unbalance': 'false',  #because training data is unbalance (replaced with scale_pos_weight)
        'num_leaves': 63,  # we should let it be smaller than 2^(max_depth)
        'max_depth': 14,  # -1 means no limit
        'min_child_samples': 35,  # Minimum number of data need in a child(min_data_in_leaf)
        'bagging_fraction':0.9429,
        'max_bin': 255,  # Number of bucketed bin for feature values
        'subsample': 0.5,  # Subsample ratio of the training instance.
       #'subsample_freq': 0.5,  # frequence of subsample, <=0 means no enable
        'feature_fraction': 0.5,  # Subsample ratio of columns when constructing each tree.
        'min_child_weight': 51.76,  # Minimum sum of instance weight(hessian) needed in a child(leaf)
        'min_split_gain': 0.07253,  # lambda_l1, lambda_l2 and min_gain_to_split to regularization
        'reg_alpha': 1.43,  # L1 regularization term on weights
        'reg_lambda': 8.33,  # L2 regularization term on weights
        'nthread': 8,
        'verbose': 0,
        'random_state':42
    }

In [ ]:
def writefiles(model,CV,sub_preds,feature_importance_df=None,IDS=None):
    CV = str(CV)
    name = model+"_submit_"+CV+".csv"
    name1 = model+"_featImp_"+CV+".csv"
    sample_submit = pd.DataFrame()
    sample_submit['impression_id'] = IDS
    sample_submit['is_click'] = sub_preds
    sample_submit.to_csv('lgbm-0.747991659378651.csv',index=False)
    if feature_importance_df is not None:
        feature_importance_df.to_csv(name1,index=False)


In [ ]:
train.head()

In [ ]:
target = train['is_click']
IDS = test['impression_id']
train.drop(['is_click','impression_id','impression_time'],1,inplace=True)
test.drop(['impression_id','impression_time'],1,inplace=True)

feature_importance_df_lgbm,sub_preds_lgbm,oof_preds_lgbm,CV_lgbm = runLGBMWithKF(train,test,target,params,5,'auto')
writefiles('lgbm-747991659378651',CV_lgbm,sub_preds_lgbm,feature_importance_df=feature_importance_df_lgbm,IDS=IDS)

In [ ]:
pd.set_option('display.max_rows',100)
_ = pd.DataFrame(feature_importance_df_lgbm[['feature','importance']].groupby('feature')['importance'].mean().sort_values(ascending=False))
_.head(100)

In [ ]:
from IPython.display import FileLink, FileLinks
FileLinks('.')